In [1]:
import os
import glob
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import joblib
from statsmodels.tsa.arima.model import ARIMA as ARIMA
from arch import arch_model
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)
pd.options.mode.chained_assignment = None

In [14]:
x=pd.read_csv('train.csv')

In [46]:
test_df = pd.read_csv('test.csv', dtype = dtypes)
test_df.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'time_id', 'row_id', 'currently_scored'],
      dtype='object')

In [19]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

def mae(y_true, y_pred):
    return np.abs(y_true - y_pred).mean()

def train_and_evaluate_ARIMA(train):
    # Initialize variables to store the best order for each stock
    best_orders = {}

    # Iterate over unique stock IDs
    unique_stock_ids = train['stock_id'].unique()
    for stock_id in unique_stock_ids:
        print(f"\nTraining and evaluating for Stock ID: {stock_id}")

        # Filter data for the current stock ID
        train_data = train[train['stock_id'] == stock_id]
        date_ids_train = train_data[['date_id', 'target']].reset_index(drop=True)

        # Split data into training and testing
        train_data = date_ids_train[date_ids_train['date_id'] <= 477]
        test_data = date_ids_train[date_ids_train['date_id'] > 477]

        print("Training Set Date Range: {} -- {}".format(train_data['date_id'].min(), train_data['date_id'].max()))
        print("Testing Set Date Range: {} -- {}".format(test_data['date_id'].min(), test_data['date_id'].max()))

        train_data = train_data.drop(['date_id'], axis=1)
        test_data = test_data.drop(['date_id'], axis=1)

        best_mae = float('inf')
        best_order = None

        # Experiment with different orders
        for p in range(3):
            for d in range(2):
                for q in range(7):
                    order = (p, d, q)
                    model = ARIMA(train_data, order=order)
                    fitted = model.fit()
                    fc = fitted.forecast(len(test_data))
                    score = mae(fc.values, test_data.values)

                    print(f"Order {order}: MAE = {score}")

                    if score < best_mae:
                        best_mae = score
                        best_order = order

        print(f'Best Order for Stock ID {stock_id}: {best_order}')
        print(f'Best MAE for Stock ID {stock_id}: {best_mae}')

        # Store the best order for the current stock ID
        best_orders[stock_id] = best_order

    return best_orders

# Assuming 'train.csv' contains the training data
train_df = pd.read_csv('train.csv')

# Find the best orders for each stock
best_orders = train_and_evaluate_ARIMA(train_df)

# Display the best orders for each stock
print("\nBest Orders for Each Stock:")
for stock_id, order in best_orders.items():
    print(f"Stock ID {stock_id}: {order}")


Training and evaluating for Stock ID: 0
Training Set Date Range: 0 -- 477
Testing Set Date Range: 478 -- 480
Order (0, 0, 0): MAE = 3.298011112594054
Order (0, 0, 1): MAE = 3.2992870662231075
Order (0, 0, 2): MAE = 3.300100554274874
Order (0, 0, 3): MAE = 3.299563148075506
Order (0, 0, 4): MAE = 3.2983591844579445
Order (0, 0, 5): MAE = 3.301060005775331
Order (0, 0, 6): MAE = 3.300839165087705
Order (0, 1, 0): MAE = 3.2930374057151512
Order (0, 1, 1): MAE = 3.274576904991574
Order (0, 1, 2): MAE = 3.2774114541344566
Order (0, 1, 3): MAE = 3.320269509225676
Order (0, 1, 4): MAE = 3.2995825179617744
Order (0, 1, 5): MAE = 3.2983876979715334
Order (0, 1, 6): MAE = 3.301146643839144
Order (1, 0, 0): MAE = 3.2979037866156613
Order (1, 0, 1): MAE = 3.2980783393394963
Order (1, 0, 2): MAE = 3.2981369438124823
Order (1, 0, 3): MAE = 3.2978259087657555
Order (1, 0, 4): MAE = 3.2984262882945727
Order (1, 0, 5): MAE = 3.3008644368584825
Order (1, 0, 6): MAE = 3.300892047326209
Order (1, 1, 0): 

/Users/alikhan/anaconda3/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Order (2, 0, 6): MAE = 3.3034719432961013
Order (2, 1, 0): MAE = 3.2755478346133877
Order (2, 1, 1): MAE = 3.2981038348335834
Order (2, 1, 2): MAE = 3.2986064962868715
Order (2, 1, 3): MAE = 3.2985338092433016
Order (2, 1, 4): MAE = 3.2987780860468554


/Users/alikhan/anaconda3/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Order (2, 1, 5): MAE = 3.3000938472891517


...

Order (2, 1, 4): MAE = 6.939587015703249


/Users/alikhan/anaconda3/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Order (2, 1, 5): MAE = 6.947878250751522


/Users/alikhan/anaconda3/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Order (2, 1, 6): MAE = 6.946850948981348
Best Order for Stock ID 102: (0, 1, 3)
Best MAE for Stock ID 102: 6.83325163952016

Best Orders for Each Stock:
Stock ID 0: (0, 1, 1)
Stock ID 1: (0, 1, 3)
Stock ID 2: (0, 0, 0)
Stock ID 3: (0, 1, 4)
Stock ID 4: (0, 1, 5)
Stock ID 5: (1, 0, 0)
Stock ID 6: (0, 0, 0)
Stock ID 7: (0, 0, 0)
Stock ID 8: (0, 0, 0)
Stock ID 9: (0, 1, 5)
Stock ID 10: (0, 0, 0)
Stock ID 11: (0, 0, 0)
Stock ID 12: (0, 1, 4)
Stock ID 13: (0, 0, 0)
Stock ID 14: (2, 1, 5)
Stock ID 15: (0, 0, 0)
Stock ID 16: (0, 0, 0)
Stock ID 17: (0, 0, 0)
Stock ID 18: (0, 0, 0)
Stock ID 19: (0, 0, 0)
Stock ID 20: (0, 1, 3)
Stock ID 21: (1, 1, 4)
Stock ID 22: (0, 1, 4)
Stock ID 23: (1, 1, 4)
Stock ID 24: (0, 1, 3)
Stock ID 25: (0, 1, 5)
Stock ID 26: (0, 0, 0)
Stock ID 27: (1, 1, 1)
Stock ID 28: (1, 1, 4)
Stock ID 29: (0, 1, 3)
Stock ID 30: (0, 1, 5)
Stock ID 31: (0, 0, 0)
Stock ID 32: (0, 0, 0)
Stock ID 33: (0, 1, 3)
Stock ID 34: (0, 1, 5)
Stock ID 35: (0, 1, 4)
Stock ID 36: (0, 1, 2)
Stock 